## Introduction
Literary authors have a wide variety of styles, contexts, and characters they describe in their works. Many of these styles are repeatable, demonstrating identifiable patterns across their bodies of work. These styles can evolve over time, changing as the lives of the authors themselves evolve. 

## Setup & Initialize Your Resources

Let's start by specifying:

The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting. If you don't specify a bucket, SageMaker SDK will create a default bucket following a pre-defined naming convention in the same region.
The IAM role ARN used to give SageMaker access to your data. It can be fetched using the get_execution_role method from sagemaker python SDK.

In [1]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

sess = sagemaker.Session()

role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = sess.default_bucket() # Replace with your own bucket name if needed
print(bucket)
prefix = 'blazingtext/supervised'

arn:aws:iam::023375022819:role/service-role/AmazonSageMaker-ExecutionRole-20181029T121824


INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-023375022819


sagemaker-us-east-1-023375022819


## Data Preperation

The data has been processed from MP3 -> Transcribe -> JSON -> CSV that we want to train the text classification model. BlazingText expects a single preprocessed text file with space separated tokens and each line of the file should contain a single sentence and the corresponding label(s) prefixed by "__label__".

Lets train the text classification model by Jane Austen and Charles Dickens. We will be Training the BlazingText model for supervised text classification.

Need to determine what length of document you use to train the model. This means that you will need to format the Transcribe results into a both data type that BlazingText can read, and a data type that intuitively lends itself well to the author classification problem. We will do this sentence-by-sentence. 

List the bucket contents

In [11]:
import boto3, os

s3 = boto3.resource('s3')

for obj in s3.Bucket(name='scientist.48.book.on.tape').objects.all():
    print(os.path.join(obj.bucket_name, obj.key))

scientist.48.book.on.tape/authors/
scientist.48.book.on.tape/mp3Audio/
scientist.48.book.on.tape/mp3Audio/Austen/
scientist.48.book.on.tape/mp3Audio/Austen/love_freindship_cs_librivox_64kb_mp3/love-freindship_1_austen_64kb.mp3
scientist.48.book.on.tape/mp3Audio/Austen/love_freindship_cs_librivox_64kb_mp3/love-freindship_2_austen_64kb.mp3
scientist.48.book.on.tape/mp3Audio/Austen/love_freindship_cs_librivox_64kb_mp3/love-freindship_3_austen_64kb.mp3
scientist.48.book.on.tape/mp3Audio/Austen/solo_pride_librivox_64kb_mp3/prideandprejudice_01-04_austen_apc_64kb.mp3
scientist.48.book.on.tape/mp3Audio/Austen/solo_pride_librivox_64kb_mp3/prideandprejudice_05-06_austen_apc_64kb.mp3
scientist.48.book.on.tape/mp3Audio/Austen/solo_pride_librivox_64kb_mp3/prideandprejudice_07-08_austen_apc_64kb.mp3
scientist.48.book.on.tape/mp3Audio/Austen/solo_pride_librivox_64kb_mp3/prideandprejudice_09-10_austen_apc_64kb.mp3
scientist.48.book.on.tape/mp3Audio/Austen/solo_pride_librivox_64kb_mp3/prideandprejudic

Read in the transcribe JSON from S3, convert to CSV per sentence

Get the CSV and break down by sentence per line

Next we will print the labels file (authors.txt) to see all possible labels followed by creating an index to label mapping.

In [2]:
!cat authors.txt

Jane Austen
Charles Dickens

The following code creates the mapping from integer indices to class label which will later be used to retrieve the actual class name during inference.

In [4]:
index_to_label = {} 
with open("authors.txt") as f:
    for i,label in enumerate(f.readlines()):
        index_to_label[str(i+1)] = label.strip()
print(index_to_label)

{'1': 'Jane Austen', '2': 'Charles Dickens'}


## Data Preprocessing

We need to preprocess the training data into space separated tokenized text format which can be consumed by BlazingText algorithm. Also, as mentioned previously, the class label(s) should be prefixed with __label__ and it should be present in the same line along with the original sentence. We'll use nltk library to tokenize the input sentences from DBPedia dataset.

Download the nltk tokenizer and other libraries

In [5]:
from random import shuffle
import multiprocessing
from multiprocessing import Pool
import csv
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Define Method transform_instance(row)

In [6]:
def transform_instance(row):
    cur_row = []
    label = "__label__" + index_to_label[row[0]]  #Prefix the index-ed label with __label__
    cur_row.append(label)
    cur_row.extend(nltk.word_tokenize(row[1].lower()))
    cur_row.extend(nltk.word_tokenize(row[2].lower()))
    return cur_row

The transform_instance will be applied to each data instance in parallel using python's multiprocessing module

Define Method preprocess(input, output, keep)

In [7]:
def preprocess(input_file, output_file, keep=1):
    all_rows = []
    with open(input_file, 'r') as csvinfile:
        csv_reader = csv.reader(csvinfile, delimiter=',')
        for row in csv_reader:
            all_rows.append(row)
    shuffle(all_rows)
    all_rows = all_rows[:int(keep*len(all_rows))]
    pool = Pool(processes=multiprocessing.cpu_count())
    transformed_rows = pool.map(transform_instance, all_rows)
    pool.close() 
    pool.join()
    
    with open(output_file, 'w') as csvoutfile:
        csv_writer = csv.writer(csvoutfile, delimiter=' ', lineterminator='\n')
        csv_writer.writerows(transformed_rows)

## Training

Now that we are done with all the setup that is needed, we are ready to train our object detector. To begin, let us create a sageMaker.estimator.Estimator object. This estimator will launch the training job.

In [8]:
region_name = boto3.Session().region_name

In [9]:
container = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "blazingtext", "latest")
print('Using SageMaker BlazingText container: {} ({})'.format(container, region_name))

Using SageMaker BlazingText container: 811284229777.dkr.ecr.us-east-1.amazonaws.com/blazingtext:latest (us-east-1)


## Hosting / Inference

## Stop / Close the Endpoint (Optional)